# Importar Modulos

In [10]:
import pandas as pd
import numpy as np
import seaborn as sn
import plotly.express as px
import tensorflow as tf

from sklearn.model_selection import train_test_split
from IPython.display import display
import keras_tuner as kt

# Importación de Datos

In [11]:
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format

data = np.loadtxt("lp5.csv", delimiter=",", dtype=str)
data_copy = data #va de 0 a 2623 o sea 2624 datos
cont = 16
labels = []
numbers = []
features= np.zeros((164,15,6))

for i in range(len(data_copy)):
    if(cont == 16):
        labels.append(data_copy[i])
    if(cont<16):
        numbers.append(data_copy[i])
    cont -= 1
    if(cont == 0):
        cont = 16
        
cont = 0
for i in range(164):
    for j in range(15):
        for z in range(6):
            features[i][j][z] = numbers[cont][z]
        cont += 1
        
labels_df = pd.DataFrame(labels, columns=['labels','1','2','3','4','5'])
#labels_df = labels_df.loc[:,['labels']]
display(labels_df)
display(features)

,labels,1,2,3,4,5
0,normal,,,,,
1,normal,,,,,
2,normal,,,,,
3,normal,,,,,
4,normal,,,,,
...,...,...,...,...,...,...
159,collision_in_tool,,,,,
160,collision_in_tool,,,,,
161,collision_in_tool,,,,,
162,collision_in_tool,,,,,


array([[[ -2.,  -1.,  81.,   0.,  -5.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        ...,
        [ -2.,  -1.,  78.,   0.,  -5.,   0.],
        [ -3.,  -1.,  80.,   1.,  -4.,   1.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.]],

       [[  6.,  -1.,  79.,  -2.,   4.,  -3.],
        [ 42.,  -3.,  80.,   5.,  53.,   3.],
        [ -5.,   4.,  74., -15., -10.,  -1.],
        ...,
        [ -1.,  -5.,  80.,   6.,  -6.,   0.],
        [ -4.,   5.,  78., -14.,  -9.,  -4.],
        [ -4.,   1.,  80.,  -3., -12.,   5.]],

       [[ -2.,  -6.,  85.,  14.,  -5.,   2.],
        [  0.,   2.,  74.,  -7.,   1.,   0.],
        [ -4.,  -5.,  76.,   7., -11.,   4.],
        ...,
        [  0.,  -9.,  87.,  13.,  -5.,   2.],
        [ -5.,   5.,  67., -17., -16.,   7.],
        [ -6., -10.,  86.,  16., -14.,  -1.]],

       ...,

       [[-12.,  17.,   3., -19., -10.,  -4.],
        [-12.,  12.,  11., -13., -16.,  -4.],
        [ -8.,   3.,  

# Manejo de Datos y Normalización

In [12]:
classes_dict = {'normal':'1 0 0 0 0', #1
        'collision_in_tool':'0 1 0 0 0', #2
        'collision_in_part':'0 0 1 0 0', #3
        'bottom_collision':'0 0 0 1 0', #4
        'bottom_obstruction':'0 0 0 0 1'} #5

labels_df = labels_df.replace({'labels':classes_dict})
labels_df[['1', '2', '3', '4', '5']] = labels_df['labels'].str.split(' ', 4, expand= True)
labels_df = labels_df.loc[:,['1', '2', '3', '4', '5']]
labels_df[['1', '2', '3', '4', '5']]=labels_df[['1', '2', '3', '4', '5']].astype(str).astype(int)

def norm(x):
    x_min = x.min()
    x_max = x.max()
    range = x_max - x_min  #min max entre 0 y 1
    return((x-x_min)/(range))

norm_features = norm(features)
#split para entrenamiento y validacion
train_features, test_features, train_labels, test_labels  = train_test_split(norm_features, labels_df, test_size=0.2, random_state= 42)
display(train_features)
display(train_labels)

C:\Users\dave4\AppData\Local\Temp\ipykernel_19480\1747463221.py:8: FutureWarning:

In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.



array([[[0.77660263, 0.78156702, 0.79084826, 0.77055903, 0.7684006 ,
         0.77422836],
        [0.77466005, 0.78739478, 0.78717893, 0.75156486, 0.7638679 ,
         0.77206993],
        [0.77401252, 0.78804231, 0.79041658, 0.74465789, 0.76106195,
         0.77012735],
        ...,
        [0.78005612, 0.7804878 , 0.78437298, 0.78135118, 0.78113533,
         0.78027196],
        [0.78027196, 0.78091949, 0.77962443, 0.78091949, 0.78027196,
         0.78027196],
        [0.78027196, 0.78070365, 0.78286208, 0.78156702, 0.78091949,
         0.78005612]],

       [[0.78027196, 0.77940859, 0.79905029, 0.78372545, 0.77962443,
         0.78113533],
        [0.78070365, 0.78113533, 0.79667602, 0.77919275, 0.78091949,
         0.78070365],
        [0.77984028, 0.77962443, 0.79710771, 0.78221455, 0.77832938,
         0.78156702],
        ...,
        [0.78070365, 0.77876106, 0.79948198, 0.78350961, 0.77962443,
         0.78113533],
        [0.77962443, 0.78178286, 0.79516512, 0.77703432, 0.777

,1,2,3,4,5
84,0,0,1,0,0
2,1,0,0,0,0
94,0,1,0,0,0
45,0,0,1,0,0
42,0,0,1,0,0
...,...,...,...,...,...
71,0,0,1,0,0
106,0,0,0,1,0
14,1,0,0,0,0
92,0,1,0,0,0


# Creación del Modelo

In [13]:
def my_model(learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape = (15,6,1), padding='same'), # #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding= 'valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),# #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding='valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model
# input shape es 15 filas, 6 columnas y 1 canal de color
#model = model = my_model(0.01)
#model.summary()

# initialize tuner to run the model.
# using the Hyperband search algorithm


# Entrenamiento del Modelo

In [14]:
def train_model(model, features, labels, epochs, batch_size):
    history = model.fit(
        x = features,
        y = labels,
        epochs= epochs,
        batch_size= batch_size,
        validation_split= 0.25
    )
    hist= pd.DataFrame(history.history) #se guardan los valores de errores y metricas en un diccionario
    hist['epoch'] = history.epoch #los epochs se deben añadir aparte
    return hist

# Visualizaciones
### Pasos para visualizar los filtros:
##### 1. Iterar por todas las capas del modelo usando model.layers
##### 2. Si la capa actual es de convolucion se extraen los pesos y sesgos usando get_weights()
##### 3. Se normalizan los pesos de los filtros entre 0 y 1
##### 4. Se plotean los filtros para cada capa convolutional y todos los canales de color

In [15]:
## Visualización de los filtros

'''def view_filters(model):
    for layer in model.layers:
        if 'conv' in layer.name:
            filters, bias= layer.get_weights()
            print(layer.name, filters.shape)
        #normalize filter values between  0 and 1 for visualization
            f_min, f_max = filters.min(), filters.max()
            filters = (filters - f_min) / (f_max - f_min)  
            print(filters.shape[3])
            axis_x=1
        #plotting all the filters
            for i in range(filters.shape[3]):
        #for i in range(6):
            #get the filters
                filt=filters[:,:,:, i]
                plotFilters(filt)
                
def plotFilters(conv_filter):
    fig, axes = plt.subplots(1, 3, figsize=(5,5))
    axes = axes.flatten()
    for img, ax in zip( conv_filter, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
   #plt.savefig('filters.jpg')
    plt.show()
## Visualización de las curvas de pérdida'''

def loss_curves(history):
    hist = history
    labels = {"loss":"Training Loss", "val_loss":"Validation Loss"}
    hist.rename(columns = labels, inplace = True)
    
    fig = px.line(hist, x='epoch', y=['Training Loss', 'Validation Loss'],
                title='Gráficas de Pérdida de Entrenamiento y Evaluación',
                labels={"epoch": "Epoch", "value":"Binary Cross Entropy", "variable":"Curvas de Pérdida"},
                color_discrete_map={ # replaces default color mapping by value
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    fig.update_layout(template='plotly_white')
    fig.show()

# Se corren las funciones

In [19]:
learning_rate = 0.0001
epochs = 200
batch_size = 1
model = my_model(learning_rate)
history= train_model(model, train_features, train_labels, epochs, batch_size)
display(history)
loss_curves(history)
#view_filters(model)
display(history)

Epoch 1/200


C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5703: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?



98/98 [==============================] - 1s 3ms/step - loss: 0.6874 - accuracy: 0.2143 - val_loss: 0.6798 - val_accuracy: 0.1818
Epoch 2/200
31/98 [========>.....................] - ETA: 0s - loss: 0.6836 - accuracy: 0.1935    

C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5703: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?



98/98 [==============================] - 0s 2ms/step - loss: 0.6732 - accuracy: 0.2143 - val_loss: 0.6603 - val_accuracy: 0.1818
Epoch 3/200
98/98 [==============================] - 0s 2ms/step - loss: 0.6447 - accuracy: 0.2347 - val_loss: 0.6225 - val_accuracy: 0.1818
Epoch 4/200
98/98 [==============================] - 0s 2ms/step - loss: 0.5966 - accuracy: 0.2449 - val_loss: 0.5735 - val_accuracy: 0.2727
Epoch 5/200
98/98 [==============================] - 0s 3ms/step - loss: 0.5500 - accuracy: 0.2653 - val_loss: 0.5378 - val_accuracy: 0.2727
Epoch 6/200
98/98 [==============================] - 0s 2ms/step - loss: 0.5443 - accuracy: 0.2347 - val_loss: 0.5202 - val_accuracy: 0.2727
Epoch 7/200
98/98 [==============================] - 0s 2ms/step - loss: 0.5264 - accuracy: 0.3061 - val_loss: 0.5060 - val_accuracy: 0.2727
Epoch 8/200
98/98 [==============================] - 0s 2ms/step - loss: 0.5061 - accuracy: 0.2755 - val_loss: 0.4990 - val_accuracy: 0.2727
Epoch 9/200
98/98 [======

# Predicciones

In [17]:
predictions = model.predict(test_features)
predictions_df = pd.DataFrame(predictions, columns=['normal', 'collision_in_tool', 'collision_in_part', 'bottom_collision', 'bottom_obstruction'])
display(predictions_df)
display(test_labels)

2/2 [==============================] - 0s 0s/step


,normal,collision_in_tool,collision_in_part,bottom_collision,bottom_obstruction
0,0.239,0.129,0.358,0.133,0.141
1,0.239,0.129,0.358,0.133,0.141
2,0.239,0.129,0.358,0.133,0.141
3,0.239,0.129,0.358,0.133,0.141
4,0.239,0.129,0.358,0.133,0.141
...,...,...,...,...,...
28,0.239,0.129,0.358,0.133,0.141
29,0.239,0.129,0.358,0.133,0.141
30,0.239,0.129,0.358,0.133,0.141
31,0.239,0.129,0.358,0.133,0.141


,1,2,3,4,5
135,0,0,0,1,0
115,0,0,0,0,1
131,0,0,0,0,1
55,1,0,0,0,0
95,0,1,0,0,0
...,...,...,...,...,...
134,0,0,0,1,0
160,0,1,0,0,0
139,0,0,0,1,0
78,0,0,1,0,0
